In [1]:
import getpass
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
load_dotenv()
from langchain_pinecone import PineconeVectorStore
from pprint import pprint
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain





/Users/nishanshehadeh/opt/anaconda3/envs/WTP/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["PINECONE_API_KEY"] = os.getenv('PINECONE_API_KEY')

llm = ChatOpenAI(model="gpt-4o", streaming = True)

In [3]:
from langchain_core.messages import AIMessage, HumanMessage


In [4]:
index_name = "langchain-index"

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
vector_store = PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [5]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
    )

In [6]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise. Give a one line summary of the context metadata used at the end of your answer"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


In [7]:

question_answer_chain = create_stuff_documents_chain(llm, prompt)
#rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [8]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [9]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [10]:
for chunk in rag_chain.stream({"input": "What are Joe Biden's tax policies?", "chat_history": []}):
    print(chunk)

{'input': "What are Joe Biden's tax policies?", 'chat_history': []}
{'context': [Document(metadata={'collection': 'whbriefingroom', 'database': 'WTP', 'id': '66a90048efac37d905d60148'}, page_content='deployment and production throughout the world. On the kind of specifics of those tasks forces, I’d — task force, I’d refer you to NSC. But just, you know, broadly, this is a key area of focus for us.MODERATOR: And with that, we will conclude our call. It is embargoed until 5:00 a.m. tomorrow. As mentioned at the top, the questions and answers were on background and are attributable to “senior administration officials.”Thank you. And tune in at 7:00 p.m. tomorrow — no, 9:00 p.m. Thank you.6:24 P.M. EST Title: Press Call by National Economic Advisor Lael Brainard and Senior Administration Officials Previewing President Biden’s State of the Union Address and Economic Agenda Date Posted: March 07, 2024 Category: Press Briefings'), Document(metadata={'collection': 'whbriefingroom', 'database':

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


In [12]:
# replace with postgresql
# return memory object here
# customize for user and chat ids
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [16]:
conversational_rag_chain.invoke(
    {"input": "What are Joe Biden's tax policies?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"Joe Biden's tax policies include extending all middle-class tax cuts while ensuring that no one making less than $400,000 per year will see a tax increase. He proposes to fund these extensions through reforms targeting the wealthy and corporations to ensure they pay their fair share, thus not adding to the debt. Additionally, he opposes extending tax cuts or restoring tax breaks for those making more than $400,000 per year.\n\n**Summary of context metadata:** Information derived from FACT SHEETS and Press Briefings related to President Biden's tax policies and economic agenda."

In [14]:
conversational_rag_chain.invoke(
    {"input": "How does he do that"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'President Joe Biden plans to extend middle-class tax cuts and ensure no tax increases for those earning under $400,000 by implementing additional reforms that make the wealthy and corporations pay their fair share. He opposes tax cuts for the wealthiest 2 percent and refuses to reintroduce deductions and other tax breaks for households earning more than $400,000. These measures aim to prevent adding to the national debt.\n\nSummary of context: "FACT SHEET: The President’s Budget Cuts Taxes for Working Families and Makes Big Corporations and the Wealthy Pay Their Fair Share" from March 2024'

In [22]:
for chunk in conversational_rag_chain.stream({"input": "How does he do that"},
    config={"configurable": {"session_id": "abc123"}}):
    print(chunk)
    
    
    

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': 'How does he do that', 'chat_history': [HumanMessage(content="What are Joe Biden's tax policies?"), AIMessage(content="Joe Biden's tax policies include extending all middle-class tax cuts while ensuring that no one making less than $400,000 per year will see a tax increase. He proposes to fund these extensions through reforms targeting the wealthy and corporations to ensure they pay their fair share, thus not adding to the debt. Additionally, he opposes extending tax cuts or restoring tax breaks for those making more than $400,000 per year.\n\n**Summary of context metadata:** Information derived from FACT SHEETS and Press Briefings related to President Biden's tax policies and economic agenda."), HumanMessage(content='How does he do that'), AIMessage(content="President Biden plans to extend middle-class tax cuts and ensure no tax increases for individuals making less than $400,000 by implementing reforms that require the wealthy and corporations to pay their fair share. This 

In [ ]:
#memory = ConversationSummaryBufferMemory(llm = llm, max_token_limit=200)
